In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.src_utils import descompress_data

tf.random.set_seed(1234)

In [8]:
descompress_data("Chest_X_Ray_DataSets.zip", "chest_xray", "data")

Descompressing files
removing .zip


'Your file is descompressed. Great!!'

In [12]:
train_path = Path("data/chest_xray/train/")
test_path = Path("data/chest_xray/test/")
val_path = Path("data/chest_xray/val")

train_image_paths = list(map(lambda x: str(x), list(train_path.glob("*/*"))))
val_image_paths = list(map(lambda x: str(x), list(val_path.glob("*/*"))))

train_image_paths[:5]

['data\\chest_xray\\train\\NORMAL\\IM-0115-0001.jpeg',
 'data\\chest_xray\\train\\NORMAL\\IM-0117-0001.jpeg',
 'data\\chest_xray\\train\\NORMAL\\IM-0119-0001.jpeg',
 'data\\chest_xray\\train\\NORMAL\\IM-0122-0001.jpeg',
 'data\\chest_xray\\train\\NORMAL\\IM-0125-0001.jpeg']

In [20]:
train_length = len(train_image_paths)
val_length = len(val_image_paths)
train_length, val_length

(5216, 16)

In [27]:
labels = {"NORMAL" : 0 , "PNEUMONIA" : 1}
invested_labels = {0 : "NORMAL", 1 : "PNEUMONIA"}


def get_labels(path: str) -> int:
    return labels[path.split("\\")[-2]]

train_labels = list(map(lambda x: get_labels(x), train_image_paths))
val_lables = list(map(lambda x: get_labels(x), val_image_paths))

print(train_labels[:3])

[0, 0, 0]
